# All Pairs Shortest Paths - Floyd-Warshall Algorithm

---

## ✨ Motivation

* In some scenarios, you need the shortest path **between all pairs** of vertices, not just from a single source.
* **Dijkstra** and **Bellman-Ford** handle **single-source** shortest paths:

  * Dijkstra: Fast, but **only** for **non-negative weights**.
  * Bellman-Ford: Works with **negative weights**, but **slower**.
* If you run these algorithms for **every vertex as source**, you get all-pairs shortest paths, but it's inefficient.
* We seek a more **direct and efficient method**: the **Floyd-Warshall Algorithm**.

---

## ⚖️ Problem Definition

* Given a graph with `n` vertices (numbered 0 to n-1) and edge weights (positive or negative, but **no negative cycles**), compute the **shortest distance between every pair (i, j)**.

---

## 🔎 Intuition from Transitive Closure

* In **transitive closure**, we determine whether there's a **path** (of any length) between vertex `i` and vertex `j`.
* Floyd-Warshall adapts this concept to compute the **shortest path length** instead of just reachability.

---

## ✏️ Key Idea (Dynamic Programming)

Let:

* `SP_k[i][j]` be the shortest distance from `i` to `j` using **only** intermediate vertices from the set `{0, 1, ..., k-1}`.
* Base case (`k = 0`): only **direct edges** allowed.
* Transition:

  ```
  SP_k[i][j] = min(SP_{k-1}[i][j], SP_{k-1}[i][k-1] + SP_{k-1}[k-1][j])
  ```

---

## 📊 Algorithm Steps

1. **Initialization**:

   * Let `dist[i][j] = weight of edge (i, j)` if it exists, else `inf`.
   * `dist[i][i] = 0` for all `i`.

2. **Main Loop**:

   ```python
   for k in range(n):
       for i in range(n):
           for j in range(n):
               if dist[i][k] + dist[k][j] < dist[i][j]:
                   dist[i][j] = dist[i][k] + dist[k][j]
   ```

3. **After loop ends**:

   * `dist[i][j]` contains the length of the shortest path from `i` to `j`.
   * If any `dist[i][i] < 0`, a **negative weight cycle** exists.

---

## 🔄 Example

Graph (with edge weights):

```
    0 --> 1 (3)
    0 --> 2 (8)
    1 --> 2 (-2)
    2 --> 0 (4)
```

Initial `dist` matrix:

```
[ [0, 3, 8],
  [inf, 0, -2],
  [4, inf, 0] ]
```

Update using `k = 0`, then `k = 1`, then `k = 2`...

Final `dist` gives shortest path between all pairs.

---

## ⌛ Time & Space Complexity

* **Time Complexity**: ❌ `O(n^3)` (Triple nested loop over `n` vertices)
* **Space Complexity**:

  * Naively: $O(n^3)$ for all `SP_k[i][j]`
  * **Optimized**: `O(n^2)` by keeping only **previous and current** matrices.

---

## 🤔 Properties

| Property                    | Floyd-Warshall             |
| --------------------------- | -------------------------- |
| Handles negative weights?   | ✅ Yes                      |
| Detects negative cycles?    | ✅ Yes (`dist[i][i] < 0`)   |
| Handles directed graphs?    | ✅ Yes                      |
| Graph representation?       | ✅ Adjacency matrix         |
| Suitable for sparse graphs? | ❌ Not efficient for sparse |

---

## 🌐 Applications

* Route planning
* Network routing
* Transitive closure computation
* Shortest path queries between arbitrary node pairs

---

## 🚀 Summary

* Floyd-Warshall computes **all-pairs shortest paths** efficiently.
* Uses a **dynamic programming** approach by gradually including more intermediate vertices.
* Works for **graphs with negative weights**, but not with **negative cycles**.
* Time: `O(n^3)`, Space: `O(n^2)` (after optimization).

---

In [ ]:
import numpy as np

def floydwarshall(WMat):
    # Extract graph dimensions (assuming square matrix)
    rows, cols, _ = WMat.shape

    # Define a large value to represent infinity
    infinity = np.max(WMat) * rows * rows + 1

    # Initialize the shortest path matrix: SP[i][j][k] means
    # shortest path from i to j using only vertices {0,...,k-1}
    SP = np.zeros((rows, cols, cols + 1))

    # Step 1: Set all initial distances to infinity
    for i in range(rows):
        for j in range(cols):
            SP[i, j, 0] = infinity

    # Step 2: Initialize SP[i][j][0] with direct edge weights from WMat
    for i in range(rows):
        for j in range(cols):
            if WMat[i, j, 0] == 1:
                SP[i, j, 0] = WMat[i, j, 1]

    # Step 3: Dynamic Programming - build SP[i][j][k] from SP[i][j][k-1]
    for k in range(1, cols + 1):
        for i in range(rows):
            for j in range(cols):
                # Either keep previous shortest path, or update via new intermediate node k-1
                SP[i, j, k] = min(SP[i, j, k-1],
                                 SP[i, k-1, k-1] + SP[k-1, j, k-1])

    # Step 4: Return the final shortest path matrix using all intermediate vertices
    return SP[:, :, cols]

## 📝 Example Input Format (WMat)

Suppose we have 3 nodes (0, 1, 2), and these edges:

* 0 → 1 with weight 4
* 1 → 2 with weight 5
* 0 → 2 with no direct edge

You can define `WMat` like this:

```python
WMat = np.zeros((3, 3, 2))
WMat[0, 1] = [1, 4]
WMat[1, 2] = [1, 5]
```

Let me know if you want a [full working example](f) with graph input and output print.


Let's break down the `floydwarshall(WMat)` function shown in your image. This is an implementation of the **Floyd-Warshall algorithm** to compute all-pairs shortest paths using **dynamic programming**.

---

## 📦 Input: `WMat`

* It's a **weighted adjacency matrix** in a 3D format:
  `WMat[i][j] = [1, weight]` if edge `i → j` exists
  `WMat[i][j] = [0, _]` if edge `i → j` does not exist

---

## 🧠 Core Idea

This implementation computes `SP[i][j][k]`:
→ the shortest path from node `i` to node `j` **using only vertices {0, 1, ..., k−1}** as intermediate nodes.

At the end, `SP[i][j][n]` contains the shortest path from `i` to `j` through **any** intermediate vertices.

---

## 📜 Line-by-Line Explanation

```python
def floydwarshall(WMat):
```

* Start of the Floyd-Warshall function. Takes a 3D matrix `WMat` as input.

```python
    (rows, cols, x) = WMat.shape
```

* Extracts the number of vertices. Assumes a square graph (`rows == cols`).

```python
    infinity = np.max(WMat) * rows * rows + 1
```

* Sets a very large value for "infinity" (to initialize distances where no path exists).

```python
    SP = np.zeros(shape=(rows, cols, cols + 1))
```

* Initializes the 3D matrix `SP` where `SP[i][j][k]` holds shortest path from `i` to `j` using only nodes {0, ..., k−1}.

---

### ⏱ Step 1: Initialize All Distances to ∞

```python
    for i in range(rows):
        for j in range(cols):
            SP[i, j, 0] = infinity
```

* Initially, all distances are set to infinity (except self-loops, which aren’t explicitly set here but could be handled).

---

### 🧱 Step 2: Fill in Direct Edges (SP\[i]\[j]\[0])

```python
    for i in range(rows):
        for j in range(cols):
            if WMat[i, j, 0] == 1:
                SP[i, j, 0] = WMat[i, j, 1]
```

* If there is a **direct edge** from `i → j`, set that weight in `SP[i][j][0]`.

---

### 🔁 Step 3: Iterative DP — Build on Previous k−1 Values

```python
    for k in range(1, cols+1):
        for i in range(rows):
            for j in range(cols):
                SP[i, j, k] = min(SP[i, j, k-1],
                                  SP[i, k-1, k-1] + SP[k-1, j, k-1])
```

* This is the **core recurrence**:

  ```
  SP[k][i][j] = min(SP[k-1][i][j], SP[k-1][i][k-1] + SP[k-1][k-1][j])
  ```

  * Either:

    * Don't use `k-1` as an intermediate: `SP[k-1][i][j]`
    * Use `k-1` as a new intermediate: `SP[k-1][i][k-1] + SP[k-1][k-1][j]`

---

### ✅ Step 4: Return Final Distance Matrix

```python
    return(SP[:, :, cols])
```

* Returns the final `SP[i][j][n]` (last matrix), which contains the shortest path from every node `i` to every node `j`.

---

## 🧮 Time and Space Complexity

* **Time**: `O(n³)` — Triple loop for every `(i, j, k)`
* **Space**: `O(n³)` — Stores `SP[i][j][k]` for all values of `k`.
  ✅ This can be optimized to `O(n²)` by storing only current and previous slices.

---